In [1]:
import sys
sys.path.append("..")

In [2]:
import os
import numpy as np
import pandas as pd
import spotlob
import tifffile as tf
import matplotlib.pyplot as plt
import imageio as im

In [3]:
from PIL import Image

In [4]:
from spotlob.process_steps import Reader
from spotlob.parameters import NumericRangeParameter
from spotlob.spim import Spim

In [5]:
class TifReader(Reader):
    """
    Reads an image from a file as an RGB file.
    Only image format `tif` is supported.
    It uses `tifffile.memmap`.
    """

    def __init__(self, filepath):
        if os.path.exists(filepath):
            im = Image.open(filepath)
            xpix, ypix = im.size
        else:
            raise IOError(f"File {filepath} not found")
        pars = [NumericRangeParameter("width", xpix, 0, xpix), 
                NumericRangeParameter("height", ypix, 0, ypix),
                NumericRangeParameter("startingpoint", (0, 0), (0, 0), (xpix, ypix))]
        super(TifReader, self).__init__(self.partial_read, pars)
       

    def partial_read(self, filepath, x_percent, y_percent, startingpoint= (0, 0)):
        """
        Returns an array of a part of an .tif image.
        The arguments must be percentages, even the startingpoint is relative.
        Starting Point is the top-left corner.
        """
        if os.path.exists(filepath):
            im = Image.open(filepath)
            width, height = im.size
            new_xpix = width * x_percent // 100
            new_ypix = height * y_percent // 100
            rel_start_x = width * startingpoint[0] // 100
            rel_start_y = height * startingpoint[1] // 100
            return (tf.memmap(filepath, dtype= np.uint8)[rel_start_y: rel_start_y + new_ypix, \
                                                         rel_start_x: rel_start_x + new_xpix, :], \
                    {"startingpoint": startingpoint, "original_width": width, "original_height": height})
        else:
            raise IOError(f"File {filepath} not found")
    
    def preview(self, spim):
        input = spim.get_at_stage(self.input_stage).image
        return self.apply(input)

In [6]:
# Creates a testtif.tif image
width = 1000
height = 1000
array = np.zeros([height, width, 3], dtype= np.uint8)
array[:height//2,:width//2] = [255, 0, 0]
array[:height//2,width//2:] = [0, 0, 255]   
array[height//2:,:width//2] = [255, 100, 255] 
array[height//2:,width//2:] = [100, 100, 255]
im.imwrite("testtif.tif", array)

In [7]:
h = TifReader("testtif.tif")

In [8]:
%matplotlib widget
array, metadata = h.partial_read("testtif.tif", 20, 15, (40, 40))
plt.imshow(array)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
from spotlob.defaults import default_pipeline
filename = "testtif.tif"

In [10]:
gui = spotlob.make_gui(filename)

In [11]:
gui.pipeline = gui.pipeline.replaced_with(TifReader)

In [12]:
spotlob.show_gui(gui)

AttributeError: 'str' object has no attribute 'function'